In [3]:
import pandas as pd
import json
import numpy as np
from collections import OrderedDict

In [5]:
fulldf = pd.read_csv('/Users/willb/Downloads/efflearnkhan_Mar07_2019_allcorrect.csv', sep='\t')
fulldf = fulldf[pd.notnull(fulldf['datastring'])]
fulldf.reset_index(drop=True, inplace=True)
for i, row in fulldf.iterrows():
    fulldf.at[i,'datastring'] = json.loads(fulldf.loc[i]['datastring'])

In [6]:
data_series = pd.Series(fulldf['datastring'])

In [18]:
def readquestions(filename):
    questiondict = {}

    infile = open(filename, "r")
    lines = infile.read().split("\n")
    

    for qv1 in range(15):   # for each question from video 1
        questiondict[lines[6 + 8*qv1].strip(" ")] = (qv1 + 1, lines[7 + 8*qv1].strip(" "))

    for qv2 in range(15):   # for each question from video 2
        questiondict[lines[129 + 8*qv2].strip(" ")] = (qv2 + 16, lines[130 + 8 * qv2].strip(" "))
    
    for qgk in range(10):   # for each question from the general knowledge section
        questiondict[lines[251 + 7 * qgk].strip(" ")] = (qgk + 31, lines[252 + 7* qgk].strip(" "))
        
    for line in lines:
        print(line)
    
    return questiondict

qdict = readquestions('../data analysis/testvideoquestions')
for q in qdict:
    print(str(qdict.get(q)[0]) +  ". " + q)
#    print(str(qdict.get(q)[1]) + "\n")
    


Note: for each question, the correct answer is the first answer

Video 1 Questions

1. Corresponding time in video (approximate): 1:00; conceptual question

Why is the gravitational attraction between you and your computer too small for you to notice?
Neither you nor your computer has enough mass to cause a noticable gravitational attraction
You and your computer are too close for the gravitational attraction to be significant
Humans are too small to detect the force of gravity
The gravitational attraction between you and your computer is disrupted by the larger gravitational field generated by the earth

2. Corresponding time in video (approximate): 1:45; factual question

Which of the following is an example of the Weak Interaction?
A neutron in a radioactive Cesium atom is converted into a proton, leading to the release of a few particles
Light from the sun collides with a satellite orbiting Earth and exerts a small push on the satellite
Two protons bound together in a Helium nucleu

In [21]:
def grade(output, questiondict, qsets, asets):
    outfile = open(output, "w")
    for set in range(3):    # for each of the 3 sets
        for question in range(13):
            if qsets[set][question] in questiondict:    # check if we have the question
                outfile.write(str(set) + "\t" + str(questiondict[qsets[set][question]][0]) + "\t")
                if asets[set][question] == questiondict[qsets[set][question]][1]:   # check if the answer is right
                    outfile.write("1")
                else:
                    outfile.write("0")
                outfile.write("\n")
            else:
                outfile.write("error" + "\n")
    outfile.close()

In [9]:
qdict = readquestions('../data analysis/testvideoquestions')

Note: for each question, the correct answer is the first answer

Video 1 Questions

1. Corresponding time in video (approximate): 1:00; conceptual question

Why is the gravitational attraction between you and your computer too small for you to notice?
Neither you nor your computer has enough mass to cause a noticable gravitational attraction
You and your computer are too close for the gravitational attraction to be significant
Humans are too small to detect the force of gravity
The gravitational attraction between you and your computer is disrupted by the larger gravitational field generated by the earth

2. Corresponding time in video (approximate): 1:45; factual question

Which of the following is an example of the Weak Interaction?
A neutron in a radioactive Cesium atom is converted into a proton, leading to the release of a few particles
Light from the sun collides with a satellite orbiting Earth and exerts a small push on the satellite
Two protons bound together in a Helium nucleu

In [22]:
for n, sub in enumerate(data_series):
    subqs = []
    subas = []
    for trial in sub['data']:
        
        if 'responses' in trial['trialdata']:
            subas.append([i.replace("&#8217;","\'") for i in OrderedDict(json.loads(trial['trialdata']['responses'])).values()])
               
        elif type(trial['trialdata']) is list:
            subqs.append([qset['prompt'].replace("&#8217;","\'") for qset in trial['trialdata']])
            
    if n == 1:
        print(subqs[0])
        print('\n\n')
        print(subas[0])
        break
    if n == 3:
        subqs = subqs[1:]
    if len(subqs) > 3:
        print('problem with sub ' + str(n) + '. q length of ' + str(len(subqs)))
    if len(subas) > 3:
        print('problem with sub ' + str(n) + '. a length of ' + str(len(subas)))
    grade("participant" + str(n + 1) + "data.tsv", qdict, subqs, subas)
    